Things to find:<br />
1)Length of the document(i.e No.of terms after tokenization)(find average also)<br />
2)Length of each section (i.e No.of terms after tokenization)(find average also)<br />
3)How many links per document(find average also)<br />
4)links received per section(find average also)<br />
5)number of images for entire document)<br />
6)number of images per section<br />
7)number of duplicate links<br />
8)Length of the document post pre processing


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize

# Function to extract the length of a document
def get_document_length(text):
    words = word_tokenize(text)
    return len(words)

# Function to calculate the average value
def calculate_average(values):
    if len(values) > 0:
      average = sum(values) / len(values)
      return  '{:.2f}'.format(average)
    return 0

# Function to extract information from a Wikipedia link
def extract_wikipedia_info(link):
    response = requests.get(link)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract the main content of the page
    main_content = soup.find(id='mw-content-text')
    if main_content is None:
        return None

    # Extract the document length
    document_length = get_document_length(main_content.get_text())

    # Extract the length of data under each headline
    headlines = main_content.find_all('span', {'class': 'mw-headline'})
    headlines_length = []
    for headline in headlines:
        next_p = headline.find_next('p')
        if next_p is not None:
            headlines_length.append(get_document_length(next_p.get_text()))

    # Extract the number of links per document
    document_links = main_content.find_all('a')
    relevant_links = [link for link in document_links if link.get('href') and not link.get('class')]
    links_per_document = len(relevant_links)

    # Extract the number of links per section
    links_per_section = []
    sections = main_content.find_all('h2')
    for section in sections:
        section_links = section.find_next_sibling('ul').find_all('a')
        relevant_section_links = [link for link in section_links if link.get('href') and not link.get('class')]
        links_per_section.append(len(relevant_section_links))

    # Extract the number of images for the entire document
    images = main_content.find_all('img')
    num_images = len(images)

    # Extract the number of images per section
    images_per_section = []
    for section in sections:
        section_images = section.find_next_sibling('ul').find_all('img')
        images_per_section.append(len(section_images))

    # Extract the number of duplicate links
    unique_links = set(link.get('href') for link in relevant_links)
    num_duplicate_links = len(relevant_links) - len(unique_links)

    return {
        'Document Length': document_length,
        'Average Length of Data per Headline': calculate_average(headlines_length),
        'Links per Document': links_per_document,
        'Average Links per Section': calculate_average(links_per_section),
        'Number of Images for Entire Document': num_images,
        'Average Images per Section': calculate_average(images_per_section),
        'Number of Duplicate Links': num_duplicate_links
    }

# Provide the Wikipedia link for extraction
link = 'https://en.wikipedia.org/wiki/Apple_Inc.'

# Extract information from the link
result = extract_wikipedia_info(link)

# Print the results
if result is not None:
    for key, value in result.items():
        print(key + ':', value)
else:
    print("Failed to extract information from the provided link.")


Document Length: 36222
Average Length of Data per Headline: 101.73
Links per Document: 3363
Average Links per Section: 6.33
Number of Images for Entire Document: 53
Average Images per Section: 0.33
Number of Duplicate Links: 835
